# LFP example 


In [16]:

from bayes_window.generative_models import generate_fake_lfp
from bayes_window.visualization import plot_data, plot_data_slope_trials
from bayes_window import visualization
from importlib import reload


## Make fake data

In [17]:

df, df_monster, index_cols, _ = generate_fake_lfp()

In [18]:
reload(visualization)
visualization.plot_data(df=df,x='stim',y='Power',color='mouse')

alt.LayerChart(...)

In [19]:
c1=visualization.plot_data(df=df,x='stim',y='Log power').properties(width=60)
c1.facet('mouse')

alt.FacetChart(...)

In [20]:
reload(visualization)
visualization.plot_data_slope_trials(df=df,x='stim',y='Power',detail='i_trial').properties().facet(column='mouse')


alt.FacetChart(...)

In [21]:
reload(visualization)
c2=visualization.plot_data_slope_trials(df=df,x='stim',y='Log power',color=None,detail='i_trial')
(c1+c2).facet(column='mouse')

alt.FacetChart(...)

## Fit and plot

In [22]:
from bayes_window import workflow
from bayes_window import models
from bayes_window import visualization
from importlib import reload
reload(workflow)
reload(visualization)
reload(models)

bw=workflow.BayesWindow(df,y='Log power', levels=('stim', 'mouse'))
bw.fit_slopes(add_data=False, model=models.model_hier_stim_one_codition,
              do_make_change='subtract', dist_y='normal')

n(Divergences) = 0


In [23]:
bw.plot_posteriors_slopes(add_box=False, independent_axes=True)

ValueError: mean(Log power diff) encoding field is specified without a type; the type cannot be inferred because it does not match any column in the data.

alt.LayerChart(...)